In [1]:
import pip
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation
import random as rd
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector
from random import randint
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid, ChartModule, TextElement
from mesa.visualization.UserParam import UserSettableParameter
from mesa.batchrunner import BatchRunner

from mesa.visualization.TextVisualization import (
    TextData, TextGrid, TextVisualization
)

### ESCENARIO 1: Las posturas de los individuos y su capacidad de convencicimiento no están condicionadas por sus atributos personales, son aleatorias. 

In [13]:
class citizen(Agent):
    
    def __init__(self,sex,race,sClass,power,postura,pos,model):
        super().__init__(pos,model)
        self.pos = pos
        self.sex = sex
        self.race = race
        self.sClass = sClass
        self.power = power
        self.postura = postura
        self.privilegio = (sex+race+sClass)/3.8
    
    

## CADA  AGENTE DELIBERA CON SUS VECINOS Y CALCULA LA POSICIÓN DEL GRUPO COMO LA POSTURA PROMEDIO
## DE SUS VECINOS PONDERADA POR EL PODER DE CONVENCIMIENTO DE CADA UNO. LA COMPARA CON LA RACIONAL Y SI 
## ES MÁS CERCANA A LA QUE TIENE ACTUALMENTE.

##SE DETIENE A LAS MIL TICKS

    def step(self):
        posturasVecinos = []
        powerVecinos = []
        for neighbor in self.model.grid.neighbor_iter(self.pos):
            posturasVecinos.append(neighbor.postura*neighbor.power)
            powerVecinos.append(neighbor.power)
            
        posturaPromedio = sum(posturasVecinos)/sum(powerVecinos)
        
        if(abs(posturaPromedio - model.posturaRacional) <= abs(self.postura - model.posturaRacional)):
            self.postura = posturaPromedio 
            
    

        
            
    
            

In [14]:
def compute_interesGeneral(model):
    agent_interes = [citizen.postura for citizen in model.schedule.agents]
    #agent_interes = [ x for x in map(int,agent_interes) ]
    return (sum(agent_interes)/len(agent_interes))
    

In [15]:
class HabermasModel(Model):
    
           
    def __init__(self):
        
        self.running = True
        self.height = 80
        self.width = 80
        self.density = 0.8
        self.grid = SingleGrid(80, 80, torus = True)        
        self.schedule = RandomActivation(self)
        self.posturaRacional = rd.random()
                
        # Create agents
        
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
        
            
            if rd.random() < self.density:
                
                # sex 1= Mujer, 1.2=Hombre
                #Igual probabilidad de ser hombre o mujer
                sexs=[ 1, 1.2]
                sex = rd.choice(sexs)
                #Races 1 = Minoría , 1.2 = Mayoría
                #P= 0.8 de pertenecer a la mayoría
                if rd.random()<= 0.8:
                    race=1.2
                else:
                    race=1            
                    # sClass 1= baja , 1.2= Media 1.4 = alta
                    # P(baja) = 0.7 ,P(media)=0.2, p(baja)=0.1

                p = rd.random()  
                if p <= 0.7:
                    sClass=1
                elif p <=0.9:
                    sClass=1.2
                else:
                    sClass = 1.4

                power = rd.random()
                postura = rd.random()
                #posturas = [rd.random() for n in range(10)]
                a = citizen(sex,race,sClass,power,postura,(x, y),self)
                self.grid.position_agent(a, (x, y))
                self.schedule.add(a)
        
        
        #DATA collector
        self.datacollector = DataCollector(
            model_reporters= {'Interes_General': compute_interesGeneral, 'Postura_Racional': 'posturaRacional'},
            agent_reporters={"Postura": "postura", 'Privilegio':'privilegio'})  # An agent attribute
        # Avance
        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
                        
        

In [16]:
i = 0
interesGeneralInstancias = pd.DataFrame([])

for i in range (10):
    
    ###corre el modelo por 1000 ticks
    model = HabermasModel()
    for i in range(100):
        model.step()
    
    interesGeneral = model.datacollector.get_model_vars_dataframe()
    
    interesGeneralInstancias = interesGeneralInstancias.append(interesGeneral)
#concatenar  data frames

In [17]:
#interesGeneralInstancias
#d = interesGeneralInstancias.to_dict()
#print(d.keys())
interesGeneralInstancias.to_csv("interesGeneralInstancias.csv")

## ESCENARIO 2: PÚBLICOS FUERTES Y DÉBILES:  Las posturas de los individuos están condicionadas por sus atributos personales, los individuos menos privilegiados tienen posturas más cercanas a las racionales. Sin embargo, tienen una menor capacidad de convencimiento.

In [19]:
class HabermasFraserModel(Model):
    
           
    def __init__(self):
        
        self.height = 80
        self.width = 80
        self.density = 0.8
        self.grid = SingleGrid(80, 80, torus = True)        
        self.schedule = RandomActivation(self)
        self.posturaRacional = rd.random()
                
        # Create agents
        
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
        
            
            if rd.random() < self.density:
                
                # sex 1= Mujer, 1.2=Hombre
                #Igual probabilidad de ser hombre o mujer
                sexs=[ 1, 1.2]
                sex = rd.choice(sexs)
                #Races 1 = Minoría , 1.2 = Mayoría
                #P= 0.8 de pertenecer a la mayoría
                if rd.random()<= 0.8:
                    race=1.2
                else:
                    race=1            
                    # sClass 1= baja , 1.2= Media 1.4 = alta
                    # P(baja) = 0.7 ,P(media)=0.2, p(baja)=0.1

                p = rd.random()  
                if p <= 0.7:
                    sClass=1
                elif p <=0.9:
                    sClass=1.2
                else:
                    sClass = 1.4

                power = rd.uniform(0, sum([sex,race,sClass])/3.8)
                
### POSTURAS MÁS CERCANAS A RACIONALES PARA MENOS PRIVILEGIADOS 

                operacion = rd.choice(['suma', 'resta'])
                if operacion == 'suma':
                    postura = self.posturaRacional + rd.uniform(0, sum([sex,race,sClass])/3.8)
                else:
                    postura = self.posturaRacional - rd.uniform(0, sum([sex,race,sClass])/3.8)
                
                
                a = citizen(sex,race,sClass,power,postura,(x, y),self)
                self.grid.position_agent(a, (x, y))
                self.schedule.add(a)
        
        
      #DATA collector
        self.datacollector = DataCollector(
            model_reporters= {'Interes_General': compute_interesGeneral, 'Postura_Racional': 'posturaRacional'},
            agent_reporters={"Postura": "postura", 'Privilegio':'privilegio'})  # An agent attribute
        # Avance
        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

In [20]:
i = 0
interesGeneralInstancias = pd.DataFrame([])

for i in range (10):
    
    ###corre el modelo por 1000 ticks
    model = HabermasFraserModel()
    for i in range(100):
        model.step()
    
    interesGeneral = model.datacollector.get_model_vars_dataframe()
    
    interesGeneralInstancias = interesGeneralInstancias.append(interesGeneral)
#concatenar  data frames
#interesGeneralInstancias
#d = interesGeneralInstancias.to_dict()
#print(d.keys())
interesGeneralInstancias.to_csv("interesGeneralInstancias.csv")

##ESCENARIO 3: Habermas realista: Las posturas de los individuos están condicionadas por sus atributos personales,de manera que las de los menos privilegiados son menos susceptibles al error.El poder de convencimiento es aleatorio.

In [21]:
class realisticHabermasModel(Model):
    
           
    def __init__(self):
        
        self.height = 80
        self.width = 80
        self.density = 0.8
        self.grid = SingleGrid(80, 80, torus = True)        
        self.schedule = RandomActivation(self)
        self.posturaRacional = rd.random()
                
        # Create agents
        
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
        
            
            if rd.random() < self.density:
                
                # sex 1= Mujer, 1.2=Hombre
                #Igual probabilidad de ser hombre o mujer
                sexs=[ 1, 1.2]
                sex = rd.choice(sexs)
                #Races 1 = Minoría , 1.2 = Mayoría
                #P= 0.8 de pertenecer a la mayoría
                if rd.random()<= 0.8:
                    race=1.2
                else:
                    race=1            
                    # sClass 1= baja , 1.2= Media 1.4 = alta
                    # P(baja) = 0.7 ,P(media)=0.2, p(baja)=0.1

                p = rd.random()  
                if p <= 0.7:
                    sClass=1
                elif p <=0.9:
                    sClass=1.2
                else:
                    sClass = 1.4

                power = rd.random()
                
### POSTURAS MÁS CERCANAS A RACIONALES PARA MENOS PRIVILEGIADOS 

                operacion = rd.choice(['suma', 'resta'])
                if operacion == 'suma':
                    postura = self.posturaRacional + rd.uniform(0, sum([sex,race,sClass])/3.8)
                else:
                    postura = self.posturaRacional - rd.uniform(0, sum([sex,race,sClass])/3.8)
                
                
                a = citizen(sex,race,sClass,power,postura,(x, y),self)
                self.grid.position_agent(a, (x, y))
                self.schedule.add(a)
        
        
      #DATA collector
        self.datacollector = DataCollector(
            model_reporters= {'Interes_General': compute_interesGeneral, 'Postura_Racional': 'posturaRacional'},
            agent_reporters={"Postura": "postura", 'Privilegio':'privilegio'})  # An agent attribute
        # Avance
        
    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

In [23]:
i = 0
interesGeneralInstancias = pd.DataFrame([])

for i in range (10):
    
    ###corre el modelo por 1000 ticks
    model = realisticHabermasModel()
    for i in range(100):
        model.step()
    
    interesGeneral = model.datacollector.get_model_vars_dataframe()
    
    interesGeneralInstancias = interesGeneralInstancias.append(interesGeneral)
#concatenar  data frames
#interesGeneralInstancias
#d = interesGeneralInstancias.to_dict()
#print(d.keys())
interesGeneralInstancias.to_csv("interesGeneralInstancias.csv")

[0.1552260853992823, 0.2472829910646125, 0.05683924063588219, 0.09774245803657078, 0.7325660268880652, 0.3956461211201854, 0.40563932017658255, 0.9233495194493914, 0.8675313076250214, 0.7835683237507228]
